**Installation of the Packages in Google Collab**

In [ ]:
!pip install --upgrade pandas requests gspread google-auth
print("Libraries installed/updated.")

Libraries installed/updated.


**Authenticate Google Account in Colab. So, the Notebook can be access Google Drive**

In [ ]:
from google.colab import auth
auth.authenticate_user()

**Setting file paths and veryfiying Data**

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# Define the path
DRIVE_DATA_FOLDER = '/content/drive/MyDrive/Data'

# Set the full paths to your .bed and .gtf files
CAT_BED_FILE = os.path.join(DRIVE_DATA_FOLDER, 'Target_bases_covered_by_probes_Methyl_EpiPaws_Small_MTE-99922492_felCat9_250327165212.bed')
DOG_BED_FILE = os.path.join(DRIVE_DATA_FOLDER, 'Target_bases_covered_by_probes_Methyl_EpiPaws_Small_MTE-95413085_canfam3_250327165327.bed')
DOG_GTF_FILE = os.path.join(DRIVE_DATA_FOLDER, 'felCat9.ncbiRefSeq.gtf')
CAT_GTF_FILE = os.path.join(DRIVE_DATA_FOLDER, 'canFam3.ncbiRefSeq.gtf')


# Verify file existence
print("\nVerifying file paths:")
files_to_verify = [CAT_BED_FILE, DOG_BED_FILE, CAT_GTF_FILE, DOG_GTF_FILE]
all_files_found = True
for f_path in files_to_verify:
    if os.path.exists(f_path):
        print(f" Found: {f_path}")
    else:
        print(f" NOT Found: {f_path}")
        all_files_found = False

if all_files_found:
    print("\nAll required files are ready in Colab environment by mounting Drive.")
else:
    print("\nERROR: Some files are missing in your Google Drive folder. Please check the paths and filenames.")

Mounted at /content/drive
Google Drive mounted.

Verifying file paths:
 Found: /content/drive/MyDrive/Data/Target_bases_covered_by_probes_Methyl_EpiPaws_Small_MTE-99922492_felCat9_250327165212.bed
 Found: /content/drive/MyDrive/Data/Target_bases_covered_by_probes_Methyl_EpiPaws_Small_MTE-95413085_canfam3_250327165327.bed
 Found: /content/drive/MyDrive/Data/canFam3.ncbiRefSeq.gtf
 Found: /content/drive/MyDrive/Data/felCat9.ncbiRefSeq.gtf

All required files are ready in Colab environment by mounting Drive.


In [ ]:
import pandas as pd
import re

def load_bed_file(bed_file, species):
    cols = ['chrom', 'start', 'end', 'cg_id']
    bed_df = pd.read_csv(bed_file, sep='\t', header=None, names=cols)
    bed_df['species'] = species
    return bed_df

dog_bed = load_bed_file(DOG_BED_FILE, 'dog')
cat_bed = load_bed_file(CAT_BED_FILE, 'cat')

# Combine into one DataFrame
bed_combined = pd.concat([dog_bed, cat_bed], ignore_index=True)
print(f"Total entries in BED combined: {len(bed_combined)}")
bed_combined.head()

Total entries in BED combined: 929


,chrom,start,end,cg_id,species
0,chr1,56283304,56283305,cg06853836,dog
1,chr1,66203061,66203062,cg12462816,dog
2,chr1,79261414,79261415,cg27370397,dog
3,chr1,91825829,91825830,cg12172533,dog
4,chr1,106090571,106090572,cg06930757,dog


In [ ]:
# Define the path to your data folder in Google Drive
DRIVE_DATA_FOLDER = '/content/drive/MyDrive/Data'

# Define file paths for BED and GTF files
CAT_BED_FILE = os.path.join(DRIVE_DATA_FOLDER, 'Target_bases_covered_by_probes_Methyl_EpiPaws_Small_MTE-99922492_felCat9_250327165212.bed')
DOG_BED_FILE = os.path.join(DRIVE_DATA_FOLDER, 'Target_bases_covered_by_probes_Methyl_EpiPaws_Small_MTE-95413085_canfam3_250327165327.bed')
DOG_GTF_FILE = os.path.join(DRIVE_DATA_FOLDER, 'dog_genomic.gtf')
CAT_GTF_FILE = os.path.join(DRIVE_DATA_FOLDER, 'cat_genomic.gtf')

**Installation of Biopython for NCBI usage***

In [ ]:
!pip install biopython tqdm

**Downloading and Extracting GTFs from the official NCBI RefSeq GTF files for Dog And Cat from UCSC**

In [ ]:
!wget http://hgdownload.soe.ucsc.edu/goldenPath/canFam3/bigZips/genes/canFam3.ncbiRefSeq.gtf.gz
!wget http://hgdownload.soe.ucsc.edu/goldenPath/felCat9/bigZips/genes/felCat9.ncbiRefSeq.gtf.gz

--2025-08-20 19:36:18--  http://hgdownload.soe.ucsc.edu/goldenPath/canFam3/bigZips/genes/canFam3.ncbiRefSeq.gtf.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18592948 (18M) [application/x-gzip]
Saving to: ‘canFam3.ncbiRefSeq.gtf.gz’

canFam3.ncbiRefSeq. 100%[===================>]  17.73M  26.1MB/s    in 0.7s    

2025-08-20 19:36:19 (26.1 MB/s) - ‘canFam3.ncbiRefSeq.gtf.gz’ saved [18592948/18592948]

--2025-08-20 19:36:19--  http://hgdownload.soe.ucsc.edu/goldenPath/felCat9/bigZips/genes/felCat9.ncbiRefSeq.gtf.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16975000 (16M) [application/x-gzip]
Saving to: ‘felCat9.ncbi

**Unzip because the data we got is compressed one**

In [ ]:
!gunzip canFam3.ncbiRefSeq.gtf.gz
!gunzip felCat9.ncbiRefSeq.gtf.gz

In [ ]:
ls -lh *.gtf

-rw-r--r-- 1 root root 303M Jan 10  2020 canFam3.ncbiRefSeq.gtf
-rw-r--r-- 1 root root 296M Apr  1  2020 felCat9.ncbiRefSeq.gtf


**Updating and Installing the required packages**

In [ ]:
!apt-get update
!apt-get install -y bedtools

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,575 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,271 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,246 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,191 kB]
Get:14 

In [ ]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.3 MB/s eta 0:00:00


**NEW SCRIPT FOR GENE ONTOLOGY**

In [ ]:
!pip install Bio

In [ ]:
"""
BED–GTF Annotation + NCBI Enrichment (Dog/CanFam3, Cat/FelCat9)
----------------------------------------------------------------
- Loads dog/cat BEDs (chrom, start, end, cg_id) and RefSeq GTFs
- Annotates each BED interval with overlapping GTF features (gene/exon/CDS…)
- Maps gene_name -> NCBI Gene ID (Entrez) and fetches descriptions/summaries
- Adds Gene Ontology (GO) terms from NCBI gene2go (BP/MF/CC)
- Exports a tidy Excel file for downstream analysis
"""

# Imports
import os, re, time, io, gzip, requests
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
from Bio import Entrez

# Configuration
Entrez.email = "hosala@epipaws.com"

OUTPUT_FILE = "/content/drive/MyDrive/Data/GO.xlsx"

# Input BED files (dog & cat probe intervals)
BED_FILES = {
    "dog": "/content/drive/MyDrive/Data/Target_bases_covered_by_probes_Methyl_EpiPaws_Small_MTE-95413085_canfam3_250327165327.bed",
    "cat": "/content/drive/MyDrive/Data/Target_bases_covered_by_probes_Methyl_EpiPaws_Small_MTE-99922492_felCat9_250327165212.bed",
}

# Input GTF files (RefSeq annotation for dog & cat)
GTF_FILES = {
    "dog": "/content/canFam3.ncbiRefSeq.gtf",
    "cat": "/content/felCat9.ncbiRefSeq.gtf",
}

# NCBI Taxon IDs (more robust filters than organism names)
TAXON = {"dog": "txid9615", "cat": "txid9685"}
TAXA_NUMERIC = {"dog": 9615, "cat": 9685}

# Parsing
def load_bed(path: str, species: str) -> pd.DataFrame:
    """Load BED file into DataFrame with: chrom, start, end, cg_id, species."""
    df = pd.read_csv(path, sep="\t", header=None, names=["chrom", "start", "end", "cg_id"])
    df["species"] = species
    return df

def parse_gtf(file: str) -> pd.DataFrame:
    """
    Parse GTF to extract: chrom, start, end, strand, feature, gene_id, gene_name.
    We read all features (not just 'gene') so we can report full overlap context.
    """
    rows = []
    with open(file) as f:
        for line in f:
            if line.startswith("#"):
                continue
            parts = line.rstrip("\n").split("\t")
            if len(parts) != 9:
                continue
            chrom, src, feature, start, end, score, strand, frame, attr = parts
            start, end = int(start), int(end)

            # parsing (key "value"; pairs)
            attrs = dict(re.findall(r'(\S+)\s+"([^"]+)"', attr))
            gene_id = attrs.get("gene_id", "")
            gene_name = attrs.get("gene_name", attrs.get("gene", ""))

            rows.append(
                {
                    "chrom": chrom,
                    "start": start,
                    "end": end,
                    "strand": strand,
                    "feature": feature,
                    "gene_id": gene_id,
                    "gene_name": gene_name,
                }
            )
    return pd.DataFrame(rows)

def annotate_bed(bed_df: pd.DataFrame, gtf_df: pd.DataFrame) -> pd.DataFrame:
    """
    For each BED interval, find overlapping GTF rows (same chrom, interval overlap).
    - feature_type: comma-joined unique types among overlaps (e.g., gene, exon, CDS)
    - gene_id/gene_name/strand: prefer a row where feature == 'gene'; else first non-empty
    """
    # Index GTF rows by chromosome for quick lookup
    gtf_by_chr = defaultdict(list)
    for _, g in gtf_df.iterrows():
        gtf_by_chr[g["chrom"]].append(g)

    annotated = []
    for _, row in tqdm(bed_df.iterrows(), total=len(bed_df), desc=f"Annotating {bed_df.iloc[0]['species']}"):
        hits = [
            g for g in gtf_by_chr.get(row["chrom"], [])
            if g["start"] <= row["end"] and g["end"] >= row["start"]
        ]

        if hits:
            # All overlapping feature types
            feature_types = sorted(set(h["feature"] for h in hits))
            feature_type_str = ",".join(feature_types)

            # Prefer 'gene' feature rows for core fields
            gene_hits = [h for h in hits if h["feature"] == "gene"]
            chosen = gene_hits[0] if gene_hits else hits[0]

            gene_id = chosen["gene_id"] or next((h["gene_id"] for h in hits if h["gene_id"]), "")
            gene_name = chosen["gene_name"] or next((h["gene_name"] for h in hits if h["gene_name"]), "")
            strand = chosen["strand"] or next((h["strand"] for h in hits if h["strand"]), "")
        else:
            feature_type_str = "intergenic"
            gene_id = gene_name = strand = ""

        annotated.append(
            {
                "chrom": row["chrom"],
                "start": row["start"],
                "end": row["end"],
                "cg_id": row["cg_id"],
                "species": row["species"],
                "gene_id": gene_id,
                "gene_name": gene_name,
                "strand": strand,
                "feature_type": feature_type_str,
            }
        )

    return pd.DataFrame(annotated)


# Entrez Gene Mapping + Summary

def esearch_gene_ids(symbols, taxon: str) -> dict:
    """
    Map gene symbols (including LOC#######) → NCBI Gene ID using Entrez.
    Strategy: exact symbol → gene name → all fields.
    Returns {symbol: gene_id or ""}.
    """
    out = {}
    for sym in tqdm(symbols, desc="Entrez search"):
        if not sym or not isinstance(sym, str):
            out[sym] = ""
            continue

        queries = [
            f'"{sym}"[Symbol] AND {taxon}[Organism:exp]',
            f'"{sym}"[Gene Name] AND {taxon}[Organism:exp]',
            f'"{sym}"[All Fields] AND {taxon}[Organism:exp]',
        ]
        gid = ""
        for q in queries:
            try:
                h = Entrez.esearch(db="gene", term=q, retmax=1)
                rec = Entrez.read(h); h.close()
                if rec.get("IdList"):
                    gid = rec["IdList"][0]
                    break
            except Exception:
                pass
            time.sleep(0.34)
        out[sym] = gid
    return out

def esummary_genes(gene_ids) -> dict:
    """
    Batch-fetch gene descriptions & summaries via Entrez ESummary.
    Returns {gid: {"ncbi_description","ncbi_summary","ncbi_url","go_terms"}}
    (GO terms left empty here; we fill from gene2go instead.)
    """
    info = {}
    ids = [gid for gid in gene_ids if gid]
    for i in range(0, len(ids), 50):
        chunk = ids[i : i + 50]
        try:
            h = Entrez.esummary(db="gene", id=",".join(chunk), retmode="xml")
            rec = Entrez.read(h); h.close()
            docs = rec["DocumentSummarySet"]["DocumentSummary"]
            for d in docs:
                gid = d.attributes["uid"]
                desc = d.get("Description", "")
                summ = d.get("Summary", "")
                info[gid] = {
                    "ncbi_description": desc,
                    "ncbi_summary": summ,
                    "go_terms": "",
                    "ncbi_url": f"https://www.ncbi.nlm.nih.gov/datasets/gene/{gid}",
                }
        except Exception:
            time.sleep(1.0)
        time.sleep(0.4)
    return info

# Gene Ontology via NCBI gene2go
def load_gene2go_for_taxa(taxa=(9615, 9685)) -> pd.DataFrame:
    """
    Download + parse NCBI gene2go; filter to requested taxa.
    Keeps: tax_id, GeneID, GO_ID, GO_term, Category (Process/Function/Component).
    """
    url = "https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz"
    r = requests.get(url, stream=True)
    r.raise_for_status()


    names = ["tax_id","GeneID","GO_ID","Evidence","Qualifier","GO_term","PubMed","Category"]

    with gzip.GzipFile(fileobj=io.BytesIO(r.content)) as gz:
        df = pd.read_csv(
            gz,
            sep="\t",
            header=None,
            names=names,
            comment="#",
            usecols=["tax_id", "GeneID", "GO_ID", "GO_term", "Category"],
            dtype={
                "tax_id": "int64",
                "GeneID": "int64",
                "GO_ID": "string",
                "GO_term": "string",
                "Category": "string",
            },
        )

    df = df[df["tax_id"].isin(taxa)].copy()
    for c in ["GO_ID", "GO_term", "Category"]:
        df[c] = df[c].fillna("")
    return df

def build_go_map(gene2go_df: pd.DataFrame) -> dict:
    """
    Build dict: {GeneID(str): "GO:xxxxxxx|Term|A; GO:yyyyyyy|Term|A; ..."}
    where A is P (Process), F (Function), C (Component).
    """
    aspect_map = {"Process": "P", "Function": "F", "Component": "C"}
    go_map = {}

    # Deduplicate per (GeneID, GO_ID) to avoid repeats
    gene2go_df = gene2go_df.drop_duplicates(["GeneID", "GO_ID"])

    for gid, sub in gene2go_df.groupby("GeneID"):
        items = []
        for _, r in sub.iterrows():
            aspect = aspect_map.get(r["Category"], r["Category"][:1] if r["Category"] else "")
            items.append(f'{r["GO_ID"]}|{r["GO_term"]}|{aspect}')
        go_map[str(gid)] = "; ".join(sorted(items))
    return go_map

# Pipeline
def main():
    all_results = []

    # 1) Prepare GO lookup once
    gene2go_df = load_gene2go_for_taxa(taxa=(TAXA_NUMERIC["dog"], TAXA_NUMERIC["cat"]))
    go_map = build_go_map(gene2go_df)

    # 2) Process each species
    for species in ["dog", "cat"]:
        print(f"\n=== Processing {species} ===")

        # Load data
        bed_df = load_bed(BED_FILES[species], species)
        gtf_df = parse_gtf(GTF_FILES[species])

        # Annotate intervals with GTF overlaps
        ann = annotate_bed(bed_df, gtf_df)

        # Map gene_name → NCBI Gene ID
        symbols = sorted(set(ann.loc[ann["gene_name"].astype(str) != "", "gene_name"].astype(str)))
        sym2gid = esearch_gene_ids(symbols, TAXON[species])
        ann["ncbi_gene_id"] = ann["gene_name"].astype(str).map(sym2gid).fillna("")

        # Fetch Entrez descriptions/summaries/urls
        gid_info = esummary_genes(sorted(set(ann["ncbi_gene_id"]) - {""}))
        ann["ncbi_description"] = ann["ncbi_gene_id"].map(
            lambda g: gid_info.get(g, {}).get("ncbi_description", "") if g else ""
        )
        ann["ncbi_summary"] = ann["ncbi_gene_id"].map(
            lambda g: gid_info.get(g, {}).get("ncbi_summary", "") if g else ""
        )
        ann["ncbi_url"] = ann["ncbi_gene_id"].map(
            lambda g: gid_info.get(g, {}).get("ncbi_url", "") if g else ""
        )

        # Attach GO terms from gene2go (by numeric GeneID key as string)
        ann["go_terms"] = ann["ncbi_gene_id"].map(lambda g: go_map.get(str(g), "") if g else "")

        all_results.append(ann)

    # 3) Combine and export
    final_df = pd.concat(all_results, ignore_index=True)

    # Column order for clean Excel
    final_cols = [
        "chrom", "start", "end", "cg_id", "species",
        "gene_id", "gene_name", "strand", "feature_type",
        "ncbi_gene_id", "ncbi_description", "ncbi_summary", "go_terms", "ncbi_url",
    ]
    for c in final_cols:
        if c not in final_df.columns:
            final_df[c] = ""
    final_df = final_df[final_cols]

    # Save
    final_df.to_excel(OUTPUT_FILE, index=False)
    print("\n Done. Saved to:", OUTPUT_FILE)
    print("Non-empty NCBI IDs:", (final_df["ncbi_gene_id"] != "").sum(), "/", len(final_df))

# Run
if __name__ == "__main__":
    main()



=== Processing dog ===


Entrez search: 100%|██████████| 235/235 [01:33<00:00,  2.51it/s]



=== Processing cat ===


Entrez search: 100%|██████████| 240/240 [01:29<00:00,  2.67it/s]



✅ Done. Saved to: /content/drive/MyDrive/Data/GO.xlsx
Non-empty NCBI IDs: 670 / 929
